In [42]:
# "Aggressive Tumor Pathology Type (AT)" A 100%

# Nonaggressive Tumor Type: Superficial BCC, Nodular BCC, SCCIS, Well-Diff SCC
# Aggressive Tumor Type: All other BCC, All other SCC, AFX, Sebaceous Carcinoma, Sweat Gland Carcinoma, Merkel Cell, Other

# sBCC = superficial basal cell carcinoma, nBCC = nodular basal cell carcinoma, SCCIS = squamous cell carcinoma in situ
# Well-Diff SCC = well-differentiated squamous cell carcinoma, AFX = atypical fibroxanthoma

# Inform GPT of what qualifies as an AT and what doesn't in system message, then ask if there is a tumor that is an AT and pass in the Addendum notes

import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

system_context = """
Your task is to determine if the following Mohs patient note contains any tumor types that qualify as Aggressive Tumor Types.
    
ONLY respond with 'Yes' or 'No'.
    
If tumor type is not explicitly named, use clinical context and wording in the note to make a best-guess classification.
    
Tumor Type Criteria:
- Nonaggressive Tumor Types: Superficial BCC, Nodular BCC, SCCIS, Well-Differentiated SCC
- Aggressive Tumor Types: All other BCC, All other SCC, AFX, Sebaceous Carcinoma, Sweat Gland Carcinoma, Merkel Cell

Only respond 'Yes' if the tumor type name or clinical description **explicitly and clearly** supports an aggressive classification.  
If the note uses vague, uncertain, or general terms — or if strong evidence of aggressiveness is missing — respond 'No'.  
When in doubt, favor 'No'.
"""

def call_gpt(query):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    
    # Payload for the request
    payload = {
    "messages" : [
            {"role": "system", "content": system_context},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.3,
      "top_p": 0.95,
      "max_tokens": 30
    }
    
    
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        print("Response Content:", response.text)
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    return response.json()["choices"][0]["message"]["content"]

In [43]:
# Use full Excel sheet
import pandas as pd
Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
Cat.head()

,Randomized Study Number,Size at Greatest Dimension (S) ***Be sure this is from the note and not the Mohs episode!,Recurrent Tumor (R),"Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)",Aggressive Tumor Pathology Type (AT),DFSP (D),Coordinated repair with ENT/oculoplastics (C),Additional C&F,Additional Excision,Multiple Sites,Business Days from Referral to First Appt,Physical Exam (PE),Assessment & Plan (A&P),Addendum (A)
0,36,0.8,Y,N,N,N,N,N,N,N,32.0,"Physical exam:\nGeneral: Well developed, well ...",1. Chronic actinic damage in the setting of hi...,Addendum 7/11/2023\n \nThe pathology results w...
1,240,0.6,Y,Y,N,N,N,N,N,N,40.0,Physical exam:\nGeneral: Well-dressed; well-no...,Procedure Orders\nBiopsy [486623795] ordered b...,Addendum 7/11/2023\nThe pathology results were...
2,32,0.3,N,N,N,N,N,N,N,N,4.0,Physical exam:\nGeneral: Well-dressed; well-no...,Assessment & Plan: \n1. Brown papule on the le...,Addendum 7/13/2023\n \nThe pathology results w...
3,111,6.5,N,N,N,N,Y,N,N,N,35.0,Physical exam:\nVital signs: There were no vit...,Assessment & Plan: \nBiopsy-proven basal cell ...,NaN
4,197,3.5,N,N,N,N,N,Y,Y,N,35.0,"Physical exam:\nGeneral: Well developed, well ...",Assessment & Plan: \n1. Chronic actinic sun d...,Addendum 7/5/2023\n \nThe pathology results we...


In [44]:
# Set options here
sample_size = len(Cat) # Change number to increase or decrease number of patients read from file
#sample_size = 10
multiple_columns = False # Set to True to use all input text columns, False to use only PE column
prioritize_column = 2   # Set to 0=PE 1=A&P 2=A to use prioritizeColumn(#), else uses callAll()
                        # If multiple_columns = False, the value of prioritize_column will be the column used
    
# Change to write GPT responses to a different txt file
txt_file = r'at_GPT_data/at_column_data_change_4.txt' # Prompt change 4 (Strict without being super strict)

# Array of indexes containing patients with notes that have problems (Some may be ok, these are from size so a location could exist but not size)
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = False # Set to True to skip bad patient notes, False to keep them in

In [45]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return "Yes" or "No" to the question
# May use only A column
import re

# Get column name to save GPT column responses to txt file
def getColumnName(column_num):
    column_name = ""
    if column_num == 0: column_name = "PE"
    elif column_num == 1: column_name = "A&P"
    elif column_num == 2: column_name = "A"
    return column_name

# Regex to get only yes/no
def cleanResponse(tmp_ans):
    check_ans = re.split(r'\s+|-', tmp_ans)  # Split by spaces or hyphens
    check_ans = [re.sub(r'[^a-zA-Z]', '', j).lower() for j in check_ans]  # Remove non-alphabetic characters, convert to lowercase
    return [j for j in check_ans if j in ['yes', 'no']] # Keep only "yes" or "no" 


# Call all 3 columns at once
def callAll(prompts):
    tmp_ans = [""] * 3
    check_ans = [""] * 3
    for i in range(3):
        tmp_ans[i] = call_gpt(prompts[i]) # Call GPT with prompt and save answer to array
        print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
        check_ans[i] = cleanResponse(tmp_ans[i])
        print(f"check_ans[{i}] = {str(check_ans[i])}")
        
        column_name = getColumnName(i)
        log_file.write(f"{column_name} Unfiltered: {tmp_ans[i]}\n")
        log_file.write(f"{column_name} Filtered: {check_ans[i]}\n")
    return tmp_ans, check_ans

# Call GPT only for a single column to save time later
def prioritizeColumn(prompts, column):
    tmp_ans = [""] * 3
    check_ans = [""] * 3
    tmp_ans[column] = call_gpt(prompts[column])
    check_ans[column] = cleanResponse(tmp_ans[column])  
    return tmp_ans, check_ans

# Check if 2 arrays are the same output, if so return one of them if it is not empty
def majorityVoteCheck(tmp_ans, check_ans):
    if check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]:
        if check_ans[0]: return tmp_ans[0]
        elif check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    elif check_ans[1] == check_ans[2]:
        if check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    else:
        return tmp_ans[1]
            
# Only call other 2 columns to check for false positives (0=PE 1=A&P 2=A)
def prioritizeA(prompts, tmp_ans, check_ans):
    if "yes" in check_ans[2]:
        for i in [0, 1]:
            tmp_ans[i] = call_gpt(prompts[i])
            print(f"tmp_ans[{i}] = {tmp_ans[i]}")
            check_ans[i] = cleanResponse(tmp_ans[i])
            print(f"check_ans[{i}] = {check_ans[i]}")
            
        if "yes" in check_ans[0] or "yes" in check_ans[1]:
            return "yes"
        else: return "no"  # A said Yes, but others disagreed, possible false positive
    else: return "no"
        
    
# Always split into 3 GPT calls, one for each column
def inputSplit(input_columns, que, index, prioritize_column):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{que} \n\n {input_string}" # Create base prompt (question + input_columns)
    log_file.write(f"Index: {index}\n") # Write index to file
    
    if(len(input_string) > 0):
        prompts = [f"{que} \n\n {col}" for col in input_columns] # Recreate prompt with question + 1 column
        
        if prioritize_column in [0, 1, 2]: # PE, A&P, A
            tmp_ans, check_ans = prioritizeColumn(prompts, prioritize_column)
            return prioritizeA(prompts, tmp_ans, check_ans)
        else:
            tmp_ans, check_ans = callAll(prompts)
            return majorityVoteCheck(tmp_ans, check_ans)

        # Count num of "yes" and "no"
        yes_count = [0] * len(input_columns)
        no_count = [0] * len(input_columns)
        
        for i in range(len(check_ans)):
            yes_count[i] = check_ans[i].count('yes')
            no_count[i] = check_ans[i].count('no')
            
        print(f"Yes counts: {yes_count}")
        print(f"No counts: {no_count}")
        
        # If no matches, return string that has the most "yes" or "no"
        if any(count > 0 for count in yes_count) or any(count > 0 for count in no_count): # Check count strings for any occurences or "yes" or "no"
            max_yes_no = [0] * len(input_columns)
            max_answer = 0
            max_i = -1
            
            # Check if string has more "yes" or "no"
            for i in range(len(input_columns)):
                max_yes_no[i] = max(yes_count[i], no_count[i])
                
            # Look for highest overall "yes" or "no" then return that string
            for i in range(len(input_columns)):
                if max_answer < max_yes_no[i]:
                    max_answer = max_yes_no[i]
                    max_i = i
            return tmp_ans[max_i]
        
        # All non "yes" or "no" answers, return empty string
        return ""
    else:
        # Somehow input_string is empty
        return "input_string empty"
    

true_results = [0 for i in range(sample_size)]
true_results_final = [0 for i in range(sample_size)]

output = [0 for i in range(sample_size)]
output_final = [0 for i in range(sample_size)]

with open(txt_file, "a") as log_file: # Open file to write column answers to for vote testing
    for i in range(0, min(sample_size, len(Cat))):
        # If skip_patients is True and current index is in skip_patients_array, move to next index
        if skip_patients and i in skip_patients_array:
            continue

        true_results[i] = Cat.at[i, 'Aggressive Tumor Pathology Type (AT)']
        que = 'Question: Does the following patient note contain an Aggressive Tumor Type?'
        ans = ""

        if multiple_columns:
            input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']]
            ans = inputSplit(input_columns, que, i, prioritize_column) # Result from GPT call
        elif not multiple_columns:
            column_names = ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']
            input_columns = Cat.at[i, column_names[prioritize_column]]
            prompt = f"{que} \n\n {input_columns}"
            
            ans = call_gpt(prompt)
            print(f"tmp_ans[{i}] = {ans}") # Change i to prioritize_column
            ans_clean = cleanResponse(ans)
            print(f"check_ans[{i}] = {ans_clean}") # Change i to prioritize_column
            
            column_name = getColumnName(prioritize_column)
            log_file.write(f"Index: {i}\n") # Write index to file
            log_file.write(f"{column_name} Unfiltered: {ans}\n")
            log_file.write(f"{column_name} Filtered: {ans_clean}\n")

        print(f"Result[{i}] = {ans}")
        output[i] = ans

tmp_ans[0] = No
check_ans[0] = ['no']
Result[0] = No
tmp_ans[1] = Yes
check_ans[1] = ['yes']
Result[1] = Yes
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[3] = No
check_ans[3] = ['no']
Result[3] = No
tmp_ans[4] = No
check_ans[4] = ['no']
Result[4] = No
tmp_ans[5] = No
check_ans[5] = ['no']
Result[5] = No
tmp_ans[6] = No
check_ans[6] = ['no']
Result[6] = No
tmp_ans[7] = No
check_ans[7] = ['no']
Result[7] = No
tmp_ans[8] = Yes
check_ans[8] = ['yes']
Result[8] = Yes
tmp_ans[9] = No
check_ans[9] = ['no']
Result[9] = No
tmp_ans[10] = Yes
check_ans[10] = ['yes']
Result[10] = Yes
tmp_ans[11] = No
check_ans[11] = ['no']
Result[11] = No
tmp_ans[12] = No
check_ans[12] = ['no']
Result[12] = No
tmp_ans[13] = No
check_ans[13] = ['no']
Result[13] = No
tmp_ans[14] = No
check_ans[14] = ['no']
Result[14] = No
tmp_ans[15] = Yes
check_ans[15] = ['yes']
Result[15] = Yes
tmp_ans[16] = No
check_ans[16] = ['no']
Result[16] = No
tmp_ans[17] = No
check_ans[17] = ['no']
Result[17] = No
tmp_ans[18

In [46]:
# Go through every GPT response and turn it into "Y" or "N" to compare
for i in range(0, min(sample_size, len(Cat))):
    if skip_patients and i in skip_patients_array:
        continue
        
    yn = output[i]
    print(f"i={i} Answer: {yn}")
    yn_real = str(true_results[i])
    
    yn = re.split(r'\s+|-', yn)  # Split by spaces or hyphens
    yn = [re.sub(r'[^a-zA-Z]', '', j).lower() for j in yn]  # Remove non-alphabetic characters, convert to lowercase
    yn = [j for j in yn if j in ['yes', 'no']] # Keep only "yes" or "no"
    
    if len(yn) == 0:
        output_final[i] = "error_2"  # No valid response detected, bad prediction
    
    yes_count = yn.count('yes')
    no_count = yn.count('no')
    
    if yes_count > no_count:
        output_final[i] = "Y"
    elif no_count > yes_count:
        output_final[i] = "N"
        
    # Check if the given input is valid
    if yn_real.strip() and yn_real.lower() != 'nan':
        if 'y' in yn_real.lower():
            true_results_final[i] = "Y" # Some are input like N/Y or Y/N/N so look for Y and if exists compare with that (increased acc by ~9%)
        else:
            true_results_final[i] = yn_real.strip()
    else:
        true_results_final[i] = "error_1" # No valid input provided in true_results, bad note
    
    print(f"Result[{i}]: ChatGPT Prediction = {output_final[i]} Answer Given = {true_results_final[i]}")

i=0 Answer: No
Result[0]: ChatGPT Prediction = N Answer Given = N
i=1 Answer: Yes
Result[1]: ChatGPT Prediction = Y Answer Given = N
i=2 Answer: No
Result[2]: ChatGPT Prediction = N Answer Given = N
i=3 Answer: No
Result[3]: ChatGPT Prediction = N Answer Given = N
i=4 Answer: No
Result[4]: ChatGPT Prediction = N Answer Given = N
i=5 Answer: No
Result[5]: ChatGPT Prediction = N Answer Given = N
i=6 Answer: No
Result[6]: ChatGPT Prediction = N Answer Given = N
i=7 Answer: No
Result[7]: ChatGPT Prediction = N Answer Given = N
i=8 Answer: Yes
Result[8]: ChatGPT Prediction = Y Answer Given = Y
i=9 Answer: No
Result[9]: ChatGPT Prediction = N Answer Given = N
i=10 Answer: Yes
Result[10]: ChatGPT Prediction = Y Answer Given = Y
i=11 Answer: No
Result[11]: ChatGPT Prediction = N Answer Given = N
i=12 Answer: No
Result[12]: ChatGPT Prediction = N Answer Given = N
i=13 Answer: No
Result[13]: ChatGPT Prediction = N Answer Given = N
i=14 Answer: No
Result[14]: ChatGPT Prediction = N Answer Given =

In [47]:
def confusionMatrix():
    TP = TN = FP = FN = 0
    for i in range(0, min((sample_size), len(Cat))):
        if output_final[i] == true_results_final[i]: # True+/True-
            if output_final[i] == "Y": # True Positive
                TP += 1
            elif output_final[i] == "N": # True Negative
                TN += 1
        elif(output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and true_results_final[i] != "error_2"): # False+/False-
            if output_final[i] == "Y": # False Positive
                FP += 1
            elif output_final[i] == "N": # False Negative
                FN += 1
    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN}

def confusionMetrics(confusion_matrix):
    TP = confusion_matrix["TP"]
    TN = confusion_matrix["TN"]
    FP = confusion_matrix["FP"]
    FN = confusion_matrix["FN"]
    
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / (TP + TN + FP + FN) if total else 0 # How often did model predict correctly
    precision = TP / (TP + FP) if (TP + FP) else 0 # When model predicted Y, how often was it correct
    recall = TP / (TP + FN) if (TP + FN) else 0 # How many of the true Y cases did model correctly identify
    F1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0 # Balance between Precision and Recall
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "F1": F1}

            
total_size = 0
correct_predictions = 0
no_response_given = 0
no_alphabetical_result_returned = 0
bad_cases_combined = 0
nrg_not_narr = 0
narr_not_nrg = 0
# nrg = no valid response given in (L) column
# narr = no alphabetical result returned from ChatGPT output

for i in range(0, min((sample_size), len(Cat))):
    if(output_final[i] == "error_2" and true_results_final[i] == "error_1"):
        bad_cases_combined += 1
        
    if output_final[i] == "error_2":
        no_alphabetical_result_returned += 1
        if true_results_final[i] != "error_1":
            narr_not_nrg += 1
            
    if true_results_final[i] == "error_1":
        no_response_given += 1
        if output_final[i] != "error_2":
            nrg_not_narr += 1
    
    if output_final[i] == true_results_final[i]:
        correct_predictions += 1
        
    total_size += 1
    
accuracy = correct_predictions / total_size
print("# of Correct Predictions = " + str(correct_predictions))
print("Sample Size = " + str(total_size))
print("Accuracy = " + "{:.2f}%".format(accuracy * 100))
print()

# Accuracy when removing cases where the Location is not given (best to look at)
no_response_given_accuracy = correct_predictions / (total_size - no_response_given)
print("Number of Notes without a Given AT = " + str(no_response_given))
print("Accuracy = " +"{:.2f}%".format(no_response_given_accuracy * 100))
print()

# Accuracy when removing cases where GPT didn't return a valid alphabetical answer (not the best to look at, could be slightly lower)
no_alphabetical_result_returned_accuracy = correct_predictions / (total_size - no_alphabetical_result_returned)
print("Number of PE/A&P/A Notes without a valid GPT Given AT = " + str(no_alphabetical_result_returned))
print("Accuracy = " + "{:.2f}%".format(no_alphabetical_result_returned_accuracy * 100))
print()

# Accuracy when removing all above bad cases (ok to look at, could be slightly lower)
bad_cases_total_accuracy = correct_predictions / (total_size - (narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Number of Combined Notes without a Given AT = " + str(narr_not_nrg + nrg_not_narr + bad_cases_combined))
print("Accuracy = " + "{:.2f}%".format(bad_cases_total_accuracy * 100))
print()

confusion_matrix = confusionMatrix()
confusion_metrics = confusionMetrics(confusion_matrix)

print("Confusion Matrix:")
for name, value in confusion_matrix.items():
    print(f"{name}: {value}")

print("\nConfusion Matrix Metrics:")
for name, value in confusion_metrics.items():
    print(f"{name}: {(value * 100):.2f}")

# of Correct Predictions = 253
Sample Size = 294
Accuracy = 86.05%

Number of Notes without a Given AT = 3
Accuracy = 86.94%

Number of PE/A&P/A Notes without a valid GPT Given AT = 0
Accuracy = 86.05%

Number of Combined Notes without a Given AT = 3
Accuracy = 86.94%

Confusion Matrix:
TP: 63
TN: 190
FP: 24
FN: 14

Confusion Matrix Metrics:
accuracy: 86.94
precision: 72.41
recall: 81.82
F1: 76.83


In [55]:
n_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i].strip() != true_results_final[i].strip() and true_results_final[i] != "error_1" and output[i] != "yes":
        n_count += 1
        print(f"i = {i}: Unfiltered ChatGPT Prediction = {output[i]}\nUnfiltered AT Given = {true_results[i]}")
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print()
        print("-"*40) # Spacer
print(f"n_count = {n_count}")
# Classifying as Yes way too often, maybe the other is what is causing the issue
# False positive is better than false negative, but early stages so there might be a fix
# 27, 102, 228 (N/Y), 237, 254 (Y/N), 286 (N/Y), 287 (N/Y)
# 126 (Y/N) 241 (N/Y)

i = 8: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 8: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 10: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 10: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 15: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 15: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 27: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 27: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 28: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 28: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 36: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y/N
i = 36: ChatGPT Prediction = N AT Given = Y

----------------------------------------
i = 40: Unfiltered ChatGPT Prediction = no
Unfiltered AT Given = Y
i = 40: ChatGPT

In [56]:
y_count = 0
for i in range(0, min((sample_size), len(Cat))):
    if output_final[i] != true_results_final[i] and true_results_final[i] != "error_1" and output[i] != "no":
        y_count += 1
        print(f"i = {i}: Unfiltered ChatGPT Prediction = {output[i]}\nUnfiltered AT Given = {true_results[i]}")
        print(f"i = {i}: ChatGPT Prediction = {output_final[i]} AT Given = {true_results_final[i]}")
        print()
        print("-"*40) # Spacer
print(f"y_count = {y_count}")

i = 9: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 9: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 16: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 16: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 17: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 17: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 32: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 32: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 37: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 37: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 47: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 47: ChatGPT Prediction = Y AT Given = N

----------------------------------------
i = 52: Unfiltered ChatGPT Prediction = yes
Unfiltered AT Given = N
i = 52: Ch